## 다항회귀
https://dsbook.tistory.com/192

* 독립변수가 단항식(x_0 ... x_n)이 아닌 2차, 3차로 표현되는 것을 다항(Polynomial) 회귀라고 한다.
* "독립변수"라는 것을 강조한 이유는 다항회귀면 비선형회귀여야 하는 것이라는 것에 대한 오해를 풀기 위해서이다. 
  * 선형회귀와 비선형회귀를 구분하는 것은 "독립변수"의 선형성이 아니라, "회귀 계수"의 선형성이다. 
  * 회귀 계수는 선형 회귀를 구현할 때와 마찬가지로 w_0, .... w_n이기 때문에, 다항 선형 회귀이다. 

In [1]:
from sklearn.preprocessing import PolynomialFeatures
import numpy as np
from sklearn.linear_model import LinearRegression

#다항식으로 변환한 단항식 생성, [[0,1], [2,3]]의 2X2 행렬 생성
X = np.arange(4).reshape(2,2)
print('일차 단항식 계수 피처:\n', X)

#degree = 2인 2차 다항식으로 변환하기 위해 PolynomialFeatures를 이용해 반환 
poly= PolynomialFeatures(degree = 2)
poly.fit(X)
poly_ftr = poly.transform(X)
print('변환된 2차 다항식 계수 피처 : \n', poly_ftr)

일차 단항식 계수 피처:
 [[0 1]
 [2 3]]
변환된 2차 다항식 계수 피처 : 
 [[1. 0. 1. 0. 0. 1.]
 [1. 2. 3. 4. 6. 9.]]


단순히 위의 형태가 아닌 임의로 원하는 다항식을 만드는 경우에는 별도의 함수를 정의해주어야 한다. 
여기에서는  y = 1 + 2 * x_1 + 3 * (x_1)^2 + 4 * (x_2)^2로 피처를 다항 피처로 변환한다. 

In [2]:
def polynomial_func(X):
    y = 1 + 2 *X[:,0] + 3 *X[:,0]**2 + 4 *X[:,1] ** 3
    return y

X = np.arange(4).reshape(2, 2)
print('일차 단항식 계수 features : \n', X)
y = polynomial_func(X)
print('삼차 다항식 결정값 : \n', y)

일차 단항식 계수 features : 
 [[0 1]
 [2 3]]
삼차 다항식 결정값 : 
 [  5 125]


단항 피처를 가지고 3차 다항 피처로 변환한 후에, 선형회귀를 이용하여 회귀계수를 찾아보자. 

In [4]:
poly_ftr

array([[ 1.,  0.,  1.,  0.,  0.,  1.,  0.,  0.,  0.,  1.],
       [ 1.,  2.,  3.,  4.,  6.,  9.,  8., 12., 18., 27.]])

In [3]:
#3차 다항식 변환 
poly_ftr = PolynomialFeatures(degree=3).fit_transform(X)
print('3차 다항식 계수 feature : \n', poly_ftr)

#Linear Regression에 3차 다항식 계수 feature와 3차 다항식 결정값으로 학습 후 회귀 계수 확인 
model = LinearRegression()
model.fit(poly_ftr, y)
print('Polynomial 회귀 계수 : \n', np.round(model.coef_, 2))
print('Polynomial 회귀 Shape : \n', model.coef_.shape)

3차 다항식 계수 feature : 
 [[ 1.  0.  1.  0.  0.  1.  0.  0.  0.  1.]
 [ 1.  2.  3.  4.  6.  9.  8. 12. 18. 27.]]
Polynomial 회귀 계수 : 
 [0.   0.18 0.18 0.36 0.54 0.72 0.72 1.08 1.62 2.34]
Polynomial 회귀 Shape : 
 (10,)


참고로 interection_only=True로 지정하면 거듭제곱이 포항된 항은 제외된다.
(즉, 거듭제곱이 있는 항 빼고 a,b,ab만 남는다.)

* 다항식의 차수가 지나치게 높아지면 오히려 더 왜곡된다. 
* 반면에 차수가 너무 작으면 데이터를 충분히 표현하지 못한다. 
* 그래서 최적의 차수를 정하는 것이 다항회귀에서 매우 중요하다. 

## Ridge와 Lasso

* 선형회귀 모델에 규제를 합쳐서 만든 모델
* 릿지
  * 계수를 제곱한 값을 기준으로 규제 적용
* 라쏘
  * 계수의 절대값을 기준으로 규제 적용
  
  Lasso는 L1 norm, Ridge는 L2 norm, ElasticNet은 L1 및 L2 norm이 혼합된 패널티

In [5]:
from sklearn.linear_model import Lasso, Ridge, ElasticNet
import numpy as np

# alphas = [10, 1, 0.1, 0.01, 0.001, 0.0001]
# 데이터셋 가정
X_train = np.random.random((10, 2))
y_train = np.random.random((10, 1))

# 라쏘, 릿지, 엘라스틱넷 적용 선형 회귀 모형 선언
model_l = Lasso(alpha = 0.1)
model_r = Ridge(alpha = 0.1)
model_e = ElasticNet(alpha = 0.1, l1_ratio = 0.5)

# 선언된 모형 학습
model_l.fit(X_train, y_train)
model_r.fit(X_train, y_train)
model_e.fit(X_train, y_train)

ElasticNet(alpha=0.1)

In [7]:
print(model_e.coef_, model_e.intercept_)

[ 0. -0.] [0.30158347]


#### alpha에 따른 회귀계수 시각화

In [ ]:
# 각 alpha에 따른 회귀 계수 값을 시각화하기 위해 5개의 열로 된 맷플롯립 축 생성  
fig , axs = plt.subplots(figsize=(18,6) , nrows=1 , ncols=5)
# 각 alpha에 따른 회귀 계수 값을 데이터로 저장하기 위한 DataFrame 생성  
coeff_df = pd.DataFrame()

# alphas 리스트 값을 차례로 입력해 회귀 계수 값 시각화 및 데이터 저장. pos는 axis의 위치 지정
for pos , alpha in enumerate(alphas) :
    ridge = Ridge(alpha = alpha)
    ridge.fit(X_data , y_target)
    # alpha에 따른 피처별 회귀 계수를 Series로 변환하고 이를 DataFrame의 컬럼으로 추가. 
    # .coef_ 를 통해 회귀 계수 추출 
    coeff = pd.Series(data=ridge.coef_ , index=X_data.columns )
    colname='alpha:'+str(alpha)
    coeff_df[colname] = coeff
    # 막대 그래프로 각 alpha 값에서의 회귀 계수를 시각화. 회귀 계수값이 높은 순으로 표현
    coeff = coeff.sort_values(ascending=False)
    axs[pos].set_title(colname)
    axs[pos].set_xlim(-3,6)
    sns.barplot(x=coeff.values , y=coeff.index, ax=axs[pos])

# for 문 바깥에서 맷플롯립의 show 호출 및 alpha에 따른 피처별 회귀 계수를 DataFrame으로 표시
plt.show()

## 성능지표

#### 1. MAE (Mean Absolute Error)

sum(|y-y_hat|)/n

* 특징
  * 모델의 예측값과 실제값 차이(Error) 를 절대값으로 변환하여 평균한 것
  * Error에 절대값을 취하기 때문에 에러의 크기 그대로 반영
  * 에러에 따른 손실이 선형적으로 올라갈 때 적합
  * 이상치가 많을 때 사용하기 적합


In [ ]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_test, y_pred)

#### 2. MSE (Mean Squared Error)

음수가 되는 오차값을 만들지 않는 방법은 절대값 말고도 제곱을하는 방법이 있다.

MAE 에서 실제 값과 예측 값의 차이(Error)를 제곱해 평균한 것

* 특징
  * 특이값이 존재하면 수치가 많이 늘어남
  * 에러에 제곱을 하기 때문에 에러가 크면 클수록 그에 따른 가중치가 높이 반영


In [ ]:
from sklearn.metrics import mean_squared_error 
mean_squared_error(y_test, y_pred)

#### 3. RMSE (Root Mean Squared Error)

MSE 값은 오차의 제곱을 구하므로 값이 커짐 그래서 MSE (Mean Squared Error)에 루트를 씌운 RMSE (Root Mean Squared Error) 값을 사용

* 특징
  * MAE와 함께 가장 일반적으로 많이 쓰이는 회귀모델 성능분석지표
  * 에러에 따른 손실이 기하 급수적으로 올라가는 상황에서 쓰기 적합

In [ ]:
from sklearn.metrics import mean_squared_error 
MSE = mean_squared_error(y_test, y_pred) 
np.sqrt(MSE)